# 使用FP-growth算法来发现频繁项集
---
使用 FP-growth算法更有效的挖掘`频繁项集`, 它只需要对数据库进行两次扫描, 而Apriori算法对于每个潜在的频繁项集都会扫描数据集判定给定模式是否频繁.
FP-growth算法将数据存储在一种称为FP树的紧凑数据结构中.FP代表频繁模式(Frequent Pattern)

使用实例: 搜索引擎自动补全查询词项.

基本过程:
- 构建FP树
- 从FP树中挖掘频繁项集

FP-growth 算法优缺点:
* 优点:  

        1. 因为 FP-growth 算法只需要对数据集遍历两次，所以速度更快。
        2. FP树将集合按照支持度降序排序，不同路径如果有相同前缀路径共用存储空间，使得数据得到了压缩。
        3. 不需要生成候选集。
        4. 比Apriori更快。
* 缺点:  

        1. FP-Tree第二次遍历会存储很多中间过程的值，会占用很多内存。
        2. 构建FP-Tree是比较昂贵的。
* 适用数据类型: 标称型数据(离散型数据)。

In [1]:
class TreeNode:
    def __init__(self, name, counts, parent):
        self.name = name  # 节点名称
        self.counts = counts  # 节点出现次数
        self.parent = parent  # 父节点
        self.children = {}  # 子节点
        # 节点链接
        self.node_link = None  # 用于连接 不同项集 的 相同的项
    
    def inc(self, counts):
        """
        增加项出现的次数
        """
        self.counts += counts
    
    def __repr__(self):
        return f"Node({self.name}: {self.counts})"
    
    def disp(self, index=1):
        """
        文本方式显示树
        """
        print(' ' * index, self.name, ' ', self.counts)
        for child in self.children.values():
            child.disp(index+1)

## FP-growth 原理
[构建FP树](https://github.com/apachecn/AiLearning/blob/master/docs/ml/12.%E4%BD%BF%E7%94%A8FP-growth%E7%AE%97%E6%B3%95%E6%9D%A5%E9%AB%98%E6%95%88%E5%8F%91%E7%8E%B0%E9%A2%91%E7%B9%81%E9%A1%B9%E9%9B%86.md#fp-growth-%E5%8E%9F%E7%90%86)

In [52]:
def load_data():
    sample = [
        ['r', 'z', 'h', 'j', 'p'],
       ['z', 'y', 'x', 'w', 'v', 'u', 't', 's'],
       ['z'],
       ['r', 'x', 'n', 'o', 's'],
    #    ['r', 'x', 'n', 'o', 's'],
       ['y', 'r', 'x', 'z', 'q', 't', 'p'],
       ['y', 'z', 'x', 'e', 'q', 's', 't', 'm']
    ]
    return sample

In [53]:
data = load_data()

In [54]:
def init_data(data):
    ret_dict = {}  # {frozenset() : 次数}
    for item in data:
        ret_dict[frozenset(item)] = ret_dict.get(frozenset(item), 0) + 1
    return ret_dict

In [55]:
class FPTree:
    def __init__(self, min_support = 1):
        # 出现的最少次数 低于此值的项会被丢弃
        self.min_support = min_support
        self.root = None
    
    def fit(self, data):
        # data = self.init_data(data)
        print("data: \n", data, "*"*30)
        self.header_table = {}
        # 1. 遍历所有的数据集合，计算所有项的支持度。
        for set_ in data:
            for item in set_:
                # set_ : frozenset(['z', 'y', 'x', 'w', 'v', 'u', 't', 's'])
                # 'z': 1(z累计次数) + 1(当前set出现次数) 
                self.header_table[item] = self.header_table.get(item, 0) + data[set_]
       
        # 2. 丢弃非频繁的项
        for key in list(self.header_table.keys()):  # 需要创建一个新的list
            if self.header_table[key] < self.min_support:
                del self.header_table[key]
#         print(head_table)
        if not self.header_table:
            return None, None
        freq_sets = set(self.header_table.keys())
        
        # 3. 基于 支持度 降序排序所有的项。
        sorted_list = sorted(self.header_table.items(), key=lambda item: item[1], reverse=True)  # [('z', 次数), ()]
        
        # 4. 所有数据集合按照得到的顺序重新整理
        # 5. 重新整理完成后，丢弃每个集合末尾非频繁的项。 
        print(sorted_list)
        new_keys = []
        for set_, freqs in data.items():
            new_keys.append([item[0] for item in sorted_list if item[0] in set_])
        print("new_keys: \n", new_keys, "*"*30)
        # [['z', 'r'], ['z', 'x', 'y', 's', 't'], ['z']]
        
        for k in self.header_table:
            # 格式化： dict{元素key: [元素次数, None]}
            self.header_table[k] = [self.header_table[k], None]
        
        # 6. 读取每个集合插入FP树中，同时用一个头部链表数据结构维护不同集合的相同项
        self.root = TreeNode('Null', 1, None)
        for key, count, new_key in zip(data.keys(), data.values(), new_keys):
            # 原始的frozenset, 项集合出现次数, 整理并舍弃末尾的集合
            self.update_tree(self.root, new_key, self.header_table, count)
    
    def update_header(self, node_to_test, target_node):
        """updateHeader(更新头指针，建立相同元素之间的关系，例如： 左边的r指向右边的r值，就是后出现的相同元素 指向 已经出现的元素)
        从头指针的nodeLink开始，一直沿着nodeLink直到到达链表末尾。这就是链表。
        性能：如果链表很长可能会遇到迭代调用的次数限制。
        Args:
            nodeToTest  满足minSup {所有的元素+(value, treeNode)}
            targetNode  Tree对象的子节点
        """
        # 建立相同元素之间的关系，例如： 左边的r指向右边的r值
        while (node_to_test.node_link is not None):
            node_to_test = node_to_test.node_link
        node_to_test.node_link = target_node
    
    def update_tree(self, root, new_keys, header_table, count):
        # 递归形式 完成FP树
        # print(root.children)
        if new_keys[0] in root.children:  # 取最前面一个(出现次数最多的)
            # 如果该元素在 inTree.children 这个字典中，就进行累加
            root.children[new_keys[0]].inc(count)
        else:
            # 如果不存在子节点，我们为该inTree添加子节点
            root.children[new_keys[0]] = TreeNode(new_keys[0], count, root)
            # 如果满足minSup的dict字典的value值第二位(位置)为null， 我们就设置该元素为 本节点对应的tree节点
            # 如果元素第二位不为null，我们就更新header节点, header_table表一项只对应一个节点
            # {'z': [5, Node(z: 5)], 'r': [3, None], }
            if header_table[new_keys[0]][1] is None:
                # headerTable只记录第一次节点出现的位置
                header_table[new_keys[0]][1] = root.children[new_keys[0]]
            else:
                # 本质上是修改headerTable的key对应的Tree，的nodeLink值
                self.update_header(header_table[new_keys[0]][1], root.children[new_keys[0]])
        if len(new_keys)>1:
            self.update_tree(root.children[new_keys[0]], new_keys[1:], header_table, count)
    


In [56]:
data_dict = init_data(data)
fp = FPTree(min_support=3)
fp.fit(data_dict)

data: 
 {frozenset({'z', 'h', 'r', 'p', 'j'}): 1, frozenset({'s', 'z', 'u', 'y', 'x', 't', 'w', 'v'}): 1, frozenset({'z'}): 1, frozenset({'s', 'x', 'r', 'o', 'n'}): 1, frozenset({'q', 'z', 'y', 'x', 'r', 't', 'p'}): 1, frozenset({'q', 's', 'z', 'y', 'x', 't', 'm', 'e'}): 1} ******************************
[('z', 5), ('x', 4), ('r', 3), ('s', 3), ('y', 3), ('t', 3)]
new_keys: 
 [['z', 'r'], ['z', 'x', 's', 'y', 't'], ['z'], ['x', 'r', 's'], ['z', 'x', 'r', 'y', 't'], ['z', 'x', 's', 'y', 't']] ******************************


In [57]:
fp.header_table

{'z': [5, Node(z: 5)],
 'r': [3, Node(r: 1)],
 's': [3, Node(s: 2)],
 'y': [3, Node(y: 2)],
 'x': [4, Node(x: 3)],
 't': [3, Node(t: 2)]}

In [58]:
fp.root.disp()

Null   1
   z   5
    r   1
    x   3
     s   2
      y   2
       t   2
     r   1
      y   1
       t   1
   x   1
    r   1
     s   1


In [59]:
def ascend_tree(leafNode, prefixPath):
    """ascendTree(如果存在父节点，就记录当前节点的name值)

    Args:
        leafNode   查询的节点对于的nodeTree
        prefixPath 要查询的节点值
    """
    if leafNode.parent is not None:
        prefixPath.append(leafNode.name)
        ascend_tree(leafNode.parent, prefixPath)

def find_prefix_path(base, tree_node):
    """findPrefixPath 基础数据集

    Args:
        basePat  要查询的节点值
        treeNode 查询的节点所在的当前nodeTree
    Returns:
        condPats 对非basePat的倒叙值作为key,赋值为count数
    """
    cond_pats = {}
    # 对 treeNode的link进行循环
    while tree_node is not None:
        prefix_path = []
        # 寻找该节点的父节点, 相当于找到了该节点的频繁项集
        ascend_tree(tree_node, prefix_path)
        # 排除自身这个元素，判断是否存在父元素（所以要>1, 说明存在父元素）
        if len(prefix_path) > 1:
            # 对非basePat的倒叙值作为key,赋值为count数
            # prefixPath[1:] 变frozenset后，字母就变无序了
            cond_pats[frozenset(prefix_path[1:])] = tree_node.counts
        # 递归，寻找该节点的下一个 相同值的链接节点
        tree_node = tree_node.node_link
    return cond_pats

In [60]:
print('x --->', find_prefix_path('x', fp.header_table['x'][1]))
print('z --->', find_prefix_path('z', fp.header_table['z'][1]))
print('r --->', find_prefix_path('r', fp.header_table['r'][1]))

x ---> {frozenset({'z'}): 3}
z ---> {}
r ---> {frozenset({'z'}): 1, frozenset({'x'}): 1, frozenset({'z', 'x'}): 1}


In [61]:
def mine_tree(intree, header_table, min_sup, prefix, freq_list):
    """mineTree(创建条件FP树)

    Args:
        inTree       myFPtree
        headerTable  满足minSup {所有的元素+(value, treeNode)}
        minSup       最小支持项集
        preFix       preFix为newFreqSet上一次的存储记录，一旦没有myHead，就不会更新
        freqItemList 用来存储频繁子项的列表
    """
    # 通过value进行从小到大的排序， 得到频繁项集的key
    # 最小支持项集的key的list集合
    bigL = [v[0] for v in sorted(header_table.items(), key=lambda x: x[1][0])]
    print('-----\n', sorted(header_table.items(), key=lambda p: p[1][0]))
    print('bigL=', bigL)
    # 循环遍历 最频繁项集的key，从小到大的递归寻找对应的频繁项集
    for base_pat in bigL:
        print("base_pat = ", base_pat)

        new_freq_set = prefix.copy()
        new_freq_set.add(base_pat)
        print('new_freq_set=', new_freq_set, prefix)

        freq_list.append(new_freq_set)
        print('freq_list=', freq_list)
        
        cond_patt_base = find_prefix_path(base_pat, header_table[base_pat][1])
        print("cond_patt_base=", base_pat, cond_patt_base)

        fp = FPTree(min_sup)
        fp.fit(cond_patt_base)
        my_cond_tree, my_head = fp.root, fp.header_table
        print("my head = ", my_head)
        if my_head:
            my_cond_tree.disp(1)
            print('\n\n\n')
            # 递归 myHead 找出频繁项集
            mine_tree(my_cond_tree, my_head, min_sup, new_freq_set, freq_list)
        print("\n\n\n")

In [62]:
fp.root

Node(Null: 1)

In [63]:
fp.header_table

{'z': [5, Node(z: 5)],
 'r': [3, Node(r: 1)],
 's': [3, Node(s: 2)],
 'y': [3, Node(y: 2)],
 'x': [4, Node(x: 3)],
 't': [3, Node(t: 2)]}

In [64]:
freq_list = []
mine_tree(fp.root, fp.header_table, 3, set([]), freq_list)

-----
 [('r', [3, Node(r: 1)]), ('s', [3, Node(s: 2)]), ('y', [3, Node(y: 2)]), ('t', [3, Node(t: 2)]), ('x', [4, Node(x: 3)]), ('z', [5, Node(z: 5)])]
bigL= ['r', 's', 'y', 't', 'x', 'z']
base_pat =  r
new_freq_set= {'r'} set()
freq_list= [{'r'}]
cond_patt_base= r {frozenset({'z'}): 1, frozenset({'x'}): 1, frozenset({'z', 'x'}): 1}
data: 
 {frozenset({'z'}): 1, frozenset({'x'}): 1, frozenset({'z', 'x'}): 1} ******************************
my head =  {}




base_pat =  s
new_freq_set= {'s'} set()
freq_list= [{'r'}, {'s'}]
cond_patt_base= s {frozenset({'z', 'x'}): 2, frozenset({'x', 'r'}): 1}
data: 
 {frozenset({'z', 'x'}): 2, frozenset({'x', 'r'}): 1} ******************************
[('x', 3)]
new_keys: 
 [['x'], ['x']] ******************************
my head =  {'x': [3, Node(x: 3)]}
  Null   1
   x   3




-----
 [('x', [3, Node(x: 3)])]
bigL= ['x']
base_pat =  x
new_freq_set= {'x', 's'} {'s'}
freq_list= [{'r'}, {'s'}, {'x', 's'}]
cond_patt_base= x {}
data: 
 {} ************************

In [65]:
freq_list  # 频繁项集

[{'r'},
 {'s'},
 {'s', 'x'},
 {'y'},
 {'y', 'z'},
 {'x', 'y'},
 {'x', 'y', 'z'},
 {'t'},
 {'t', 'z'},
 {'t', 'y'},
 {'t', 'y', 'z'},
 {'t', 'x'},
 {'t', 'x', 'z'},
 {'t', 'x', 'y'},
 {'t', 'x', 'y', 'z'},
 {'x'},
 {'x', 'z'},
 {'z'}]